## **Webscraping com Python no SQL Server**

In [1]:
DECLARE @pscript NVARCHAR(MAX);

/* 1 Importar os módulos python */
-- Instalar módulos no SQL Server
-- pandas, requests, bs4, html5lib, lxml

SET @pscript = N'

import pandas as pd                  # importar para criar o dataframe
import requests                      # importar para consulta web
from bs4 import BeautifulSoup as bs  # importar para ler html

'

/* 2. Obter Texto HTML BeatifulSoup */
-- Conhecer e Identificar o código HTML

SET @pscript = @pscript + N'

url = "https://www.macrotrends.net/stocks/charts/MSFT/microsoft/stock-price-history"
html_data = requests.get(url).text  # consulta para a página de internet
beautiful_soup = bs(html_data, "html5lib")  # ler e converter html em objetos

'

/* 3. Obter e identificar a tabela desejada */

SET @pscript = @pscript + N'

tabelas = beautiful_soup.find_all("table")

for id, tabela in enumerate(tabelas):
    if ("Historical Annual Stock Price Data" in str(tabela)):
        tabela_id = id      # obter o índice da tabela

'


/* 4. Cria Dataframe */

SET @pscript = @pscript + N'

table_data = pd.DataFrame(columns=["Ticker","Year", "Average", "Open", "High", "Low", "Close"]) #Inserir colunas no dataframe

'


/* 5. Preencher Dataframe com valores da tabela */

SET @pscript = @pscript + N'

for linha in tabelas[tabela_id].tbody.find_all("tr"):
    coluna = linha.find_all("td")
    if (coluna !=[]):
		year = coluna[0].text
		average = coluna[1].text
		open = coluna[2].text
		high = coluna[3].text
		low = coluna[4].text
		close = coluna[5].text
        
		table_data = table_data.append(\
        {"Ticker": "MSFT" ,\
        "Year" : year, \
        "Average" : average, \
        "Open" : open, \
        "High" : high, \
        "Low" : low, \
        "Close" : close}, ignore_index=True)

'

/* 6. Disponibilizar o resultado */

SET @pscript = @pscript + N'

OutputDataSet = table_data

'


Execute sp_execute_external_script 
@language = N'Python', 
@script = @pscript
WITH RESULT SETS( ("Ticker" varchar(10), "Year" int, "Average" float, "Open" float, "High" float, "Low" float, "Close" float) ); -- nomes das colunas


STDERR message(s) from external script: 
C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\PYTHON_SERVICES\lib\site-packages\revoscalepy\functions\RxSummary.py:4: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
 from pandas import DataFrame, Index, Panel

(36 rows affected)

Total execution time: 00:00:09.742

Ticker,Year,Average,Open,High,Low,Close
MSFT,2022,"305,1354","334,75","334,75","287,72","287,72"
MSFT,2021,"275,9408","217,69","343,11","212,25","336,32"
MSFT,2020,"193,0261","160,62","231,65","135,42","222,42"
MSFT,2019,"130,382","101,12","158,96","97,4","157,7"
MSFT,2018,"101,034","85,95","115,61","85,01","101,57"
MSFT,2017,"71,984","62,58","86,85","62,3","85,54"
MSFT,2016,"55,2593","54,8","63,62","48,43","62,14"
MSFT,2015,"46,7136","46,76","56,55","40,29","55,48"
MSFT,2014,"42,4533","37,16","49,61","34,98","46,45"
MSFT,2013,"32,4915","27,62","38,94","26,46","37,41"


## **Criar Procedimento**

In [27]:
CREATE OR ALTER PROCEDURE ObterTicker 
	@Nome varchar(MAX), --definir parametro para o ticker desejado
	@Excluir bit = 0    -- definir parametro para excluir os registros antes
AS
BEGIN


IF @Excluir = 1 
	DELETE FROM STOCK_PRICE_HISTORY

DECLARE @pscript NVARCHAR(MAX);

SET @pscript = N'

import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

url = "https://www.macrotrends.net/stocks/charts/'+ @Nome + N'/stock-price-history"
html_data = requests.get(url).text
beautiful_soup = bs(html_data, "html5lib")

tabelas = beautiful_soup.find_all("table")

for id,tabela in enumerate(tabelas):
    if ("Historical Annual Stock Price Data" in str(tabela)):
        tabela_id = id

table_data = pd.DataFrame(columns=["Ticker","Year", "Average", "Open", "High", "Low", "Close"])

for linha in tabelas[tabela_id].tbody.find_all("tr"):
    coluna = linha.find_all("td")
    if (coluna !=[]):
		year = coluna[0].text
		average = coluna[1].text
		open = coluna[2].text
		high = coluna[3].text
		low = coluna[4].text
		close = coluna[5].text
        
		table_data = table_data.append(\
        {"Ticker": "'+ LEFT(@Nome,4) + N'" ,\
        "Year" : year, \
        "Average" : average, \
        "Open" : open, \
        "High" : high, \
        "Low" : low, \
        "Close" : close}, ignore_index=True)

OutputDataSet = table_data

'

INSERT INTO STOCK_PRICE_HISTORY --Inserir o resultado na tabela
Execute sp_execute_external_script 
@language = N'Python', 
@script = @pscript


END
GO

Commands completed successfully.

Total execution time: 00:00:00.007

## **Executar Procedimento**

In [28]:
-- Obter valores da APPLE
EXEC ObterTicker 'AAPL/apple', 1;
GO

-- Obter Valores da Microsoft
EXEC ObterTicker 'MSFT/microsoft';
GO

(78 rows affected)

STDERR message(s) from external script: 
C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\PYTHON_SERVICES\lib\site-packages\revoscalepy\functions\RxSummary.py:4: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
 from pandas import DataFrame, Index, Panel

(42 rows affected)

STDERR message(s) from external script: 
C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\PYTHON_SERVICES\lib\site-packages\revoscalepy\functions\RxSummary.py:4: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
 from pandas import DataFrame, Index, Panel

(36 rows affected)

Total execution time: 00:00:07.807

## **Consultar Tabela**

In [29]:
SELECT * FROM STOCK_PRICE_HISTORY

(78 rows affected)

Total execution time: 00:00:00.058

Ticker,Year,Average,Open,High,Low,Close
AAPL,2022,"170,7912","182,01","182,01","159,22","167,3"
AAPL,2021,"140,9893","129,41","180,33","116,36","177,57"
AAPL,2020,"95,3468","75,0875","136,69","56,0925","132,69"
AAPL,2019,"52,064","39,48","73,4125","35,5475","73,4125"
AAPL,2018,"47,2634","43,065","58,0175","36,7075","39,435"
AAPL,2017,"37,6378","29,0375","44,105","29,005","42,3075"
AAPL,2016,"26,151","26,3375","29,5625","22,585","28,955"
AAPL,2015,"30,0096","27,3325","33,25","25,78","26,315"
AAPL,2014,"23,0661","19,7546","29,75","17,8494","27,595"
AAPL,2013,"16,8798","19,6082","20,3604","13,9475","20,0364"
